### Use Machine Learning to predict the probability of needing Post-Mastectom Radiotherapy
Author: Yifu (Charles) Chen

A notebook in Python Language

In [ ]:
### Import modules ###
import os
import glob
import pandas as pd
import numpy as np

In [ ]:
# Directory of DataFrames
dir_to_df = "/Users/yifu/PycharmProjects/Radiotherapy-Prediction/data/experiments/May18 Datasets for Experiments Table Debugged"
# Assume all files in directory are CSV DataFrame files
paths_to_df = glob.glob(os.path.join(dir_to_df, "*.csv"))

# Read each DataFrame file
